In [ ]:
from random import randint

def generate_superincreasing_sequence(length):
    sequence = [randint(1, 10)]
    for _ in range(length - 1):
        sequence.append(sum(sequence) + randint(sequence[-1] + 1, sequence[-1] * 2))
    return sequence

def generate_modulus(sequence):
    return sum(sequence) + randint(1, 10)

def generate_public_key(sequence, modulus):
    multiplier = randint(2, 10)
    return [(elem * multiplier) % modulus for elem in sequence]

def generate_private_key(sequence, modulus):
    inverse = pow(sum(sequence), -1, modulus)
    return [(elem * inverse) % modulus for elem in sequence]

def encrypt_message(message, public_key):
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    return sum(int(bit) * public_key[i % len(public_key)] for i, bit in enumerate(binary_message))

def decrypt_message(encrypted_message, private_key, modulus):
    total = encrypted_message
    bits = ''
    for key in reversed(private_key):
        if total >= key:
            bits = '1' + bits
            total -= key
        else:
            bits = '0' + bits
    chars = [chr(int(bits[i:i+8], 2)) for i in range(0, len(bits), 8)]
    return ''.join(chars)

def main():
    length = 8
    sequence = generate_superincreasing_sequence(length)
    modulus = generate_modulus(sequence)
    public_key = generate_public_key(sequence, modulus)
    private_key = generate_private_key(sequence, modulus)
    message = "Hello, Knapsack!"
    print("Original message:", message)
    encrypted_message = encrypt_message(message, public_key)
    print("Encrypted message:", encrypted_message)
    decrypted_message = decrypt_message(encrypted_message, private_key, modulus)
    print("Decrypted message:", decrypted_message)

if __name__ == "__main__":
    main()
